In [0]:
USE CATALOG dbdemos_vishesh;
USE SCHEMA contest_mock;

CREATE OR REPLACE VIEW contests_view AS
SELECT 
    -- Contest Information
    c.contest_id,
    c.contest_sport,
    c.contest_type,
    c.contest_create_ts,
    c.entry_amount,
    c.prize_pool,
    
    -- Contest Participation Information
    ct.user_id,
    ct.team_id,
    ct.fee_paid,
    ct.final_position,
    
    -- User Demographics
    u.created_timestamp as user_created_timestamp,
    u.origin_state,
    u.gender,
    
    -- Aggregated Wallet Information (per user)
    wt.total_wallet_amount,
    wt.total_wallet_transactions,
    wt.avg_transaction_amount,
    wt.total_net_amount,
    wt.total_discount_points,
    wt.preferred_payment_method,
    wt.payment_method_diversity

FROM contests c
LEFT JOIN contests_tx ct ON c.contest_id = ct.contest_id
LEFT JOIN users u ON ct.user_id = u.user_id
LEFT JOIN (
    -- Pre-aggregate wallet transactions per user
    SELECT 
        user_id,
        SUM(amount) as total_wallet_amount,
        COUNT(*) as total_wallet_transactions,
        AVG(amount) as avg_transaction_amount,
        SUM(net_amount) as total_net_amount,
        SUM(discount_points) as total_discount_points,
        MODE() WITHIN GROUP (ORDER BY instrument_type) as preferred_payment_method,
        COUNT(DISTINCT instrument_type) as payment_method_diversity
    FROM wallet_tx
    GROUP BY user_id
) wt ON u.user_id = wt.user_id;
